# **Class 11: Semantic Router**

Semantic Router is a superfast decision-making layer for your LLMs and agents. Rather than waiting for slow LLM generations to make tool-use decisions, we use the magic of semantic vector space to make those decisions — routing our requests using semantic meaning.

https://github.com/aurelio-labs/semantic-router

How It Works:

1. Embedding Generation: Inputs and possible actions/tools are embedded into a high-dimensional vector space using a vectorizer like OpenAI embeddings, Sentence Transformers, or custom models.

2. Similarity Computation: The router computes the similarity between the input vector and the vectors for available routes.

3. Routing Decision: Based on the highest similarity score, the Semantic Router directs the request to the best match.

In [ ]:
from semantic_router import Route
from semantic_router.encoders import OpenAIEncoder, HuggingFaceEncoder
from semantic_router import RouteLayer
import pandas as pd
from sklearn.model_selection import train_test_split

## Preprocessing

In [2]:
# Load the dataframes json files
df_loaded = pd.read_json("synthetic_messages.json")

In [3]:
X = df_loaded[['Id','Message']]
y = df_loaded['Intention'].to_list()

In [ ]:
# Split the dataset with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=0, stratify=y
)

In [5]:
# Replace "None" with None
y_train = [None if i == "None" else i for i in y_train]
y_test = [None if i == "None" else i for i in y_test]

In [6]:
# Set the column Id as the index
X_train.set_index('Id', inplace=True)
X_test.set_index('Id', inplace=True)

In [7]:
order_status_messages = []
create_order_messages = []
product_information_messages = []

for message, label in zip(X_train["Message"], y_train):
    if label == 'order_status':
        order_status_messages.append(message)
    elif label == 'create_order':
        create_order_messages.append(message)
    elif label == 'product_information':
        product_information_messages.append(message)

## Routes

We start by defining a dictionary mapping routes to example phrases that should trigger those routes.

In [8]:
order_status = Route(
    name="order_status",
    description="The user wants to know the status of their order.",
    utterances=order_status_messages,
)

create_order = Route(
    name="create_order",
    description="The user intends to place an order for a product on the Cobuy platform.",
    utterances=create_order_messages,
)

product_information = Route(
    name="product_information",
    description="The user is interested in obtaining information about a specific product available on the Cobuy platform.",
    utterances=product_information_messages,
)

Let's create a list of routes:

In [9]:
routes = [order_status, create_order, product_information]

## Semantic Router

We define a route layer using these routes and using the OpenAI encoder.

In [10]:
encoder = HuggingFaceEncoder()
hf_rl = RouteLayer(encoder=encoder, routes=routes) #aggregation = "mean", "max" or "sum". #top_k = 5

In [11]:
encoder = OpenAIEncoder()
oa_rl = RouteLayer(encoder=encoder, routes=routes) #aggregation = "mean", "max" or "sum" #top_k = 5

Now we can test it:

In [12]:
hf_rl("how's the weather today?")

RouteChoice(name=None, function_call=None, similarity_score=None)

In [13]:
oa_rl("how's the weather today?")

RouteChoice(name=None, function_call=None, similarity_score=None)

In [14]:
hf_rl.retrieve_multiple_routes("Hi! Can i get my order??")

[RouteChoice(name='create_order', function_call=None, similarity_score=0.5805390220278703)]

In [15]:
oa_rl.retrieve_multiple_routes("Hi! Can i get my order??")

[RouteChoice(name='create_order', function_call=None, similarity_score=0.6125962591497243),
 RouteChoice(name='order_status', function_call=None, similarity_score=0.5912817734453557)]

In [16]:
hf_rl.retrieve_multiple_routes("Hi! Can i get my order??")

[RouteChoice(name='create_order', function_call=None, similarity_score=0.5805390220278703)]

In [17]:
oa_rl.retrieve_multiple_routes("Hi! Can i get my order??")

[RouteChoice(name='create_order', function_call=None, similarity_score=0.6125962591497243),
 RouteChoice(name='order_status', function_call=None, similarity_score=0.5912817734453557)]

We can evaluate the performance of our route layer using the evaluate method. All we need is to pass a list of utterances and target route labels:

In [18]:
# evaluate using the default thresholds
accuracy = hf_rl.evaluate(X=X_test["Message"].to_list(), y=y_test)
print(f"Accuracy: {accuracy*100:.2f}%")

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Accuracy: 90.91%


In [19]:
# evaluate using the default thresholds
accuracy = oa_rl.evaluate(X=X_test["Message"].to_list(), y=y_test)
print(f"Accuracy: {accuracy*100:.2f}%")

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Accuracy: 81.82%


On this subset we get reasonable accuracy.

## Route Layer Optimization

Our optimization works by finding the best route thresholds for each Route in our RouteLayer. We can see the current, default thresholds by calling the get_thresholds method:

In [20]:
route_thresholds = hf_rl.get_thresholds()
print("Default route thresholds:", route_thresholds)

Default route thresholds: {'order_status': 0.5, 'create_order': 0.5, 'product_information': 0.5}


In [21]:
route_thresholds = oa_rl.get_thresholds()
print("Default route thresholds:", route_thresholds)

Default route thresholds: {'order_status': 0.5, 'create_order': 0.5, 'product_information': 0.5}


These are all preset route threshold values. Fortunately, it's very easy to optimize these — we simply call the fit method and provide our training utterances X, and target route labels y:

In [22]:
# Call the fit method
hf_rl.fit(X=X_train["Message"].to_list(), y=y_train, max_iter=500)

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Training:   0%|          | 0/500 [00:00<?, ?it/s]

In [23]:
# Call the fit method
oa_rl.fit(X=X_train["Message"].to_list(), y=y_train, max_iter=500)

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Training:   0%|          | 0/500 [00:00<?, ?it/s]

Let's see what our new thresholds look like:

In [24]:
route_thresholds = hf_rl.get_thresholds()
print("Updated route thresholds:", route_thresholds)

Updated route thresholds: {'order_status': 0.5, 'create_order': 0.5, 'product_information': 0.5}


In [25]:
route_thresholds = oa_rl.get_thresholds()
print("Updated route thresholds:", route_thresholds)

Updated route thresholds: {'order_status': 0.5, 'create_order': 0.5, 'product_information': 0.5}


These are vastly different thresholds to what we were seeing before — it's worth noting that optimal values for different encoders can vary greatly.

After training we have a final performance of:

In [26]:
accuracy = hf_rl.evaluate(X=X_test["Message"].to_list(), y=y_test)
print(f"Accuracy: {accuracy*100:.2f}%")

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Accuracy: 90.91%


In [27]:
accuracy = oa_rl.evaluate(X=X_test["Message"].to_list(), y=y_test)
print(f"Accuracy: {accuracy*100:.2f}%")

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Accuracy: 81.82%


That is much better. If we wanted to optimize this further we can focus on adding more utterances to our existing routes, analyzing where exactly our failures are, and modifying our routes around those. This extended optimzation process is much more manual, but with it we can continue optimizing routes to get even better performance.

In [30]:
for (index, row), label in zip(X_test.iterrows(), y_test):
    message = row["Message"]
    prediction = hf_rl(message)

    if prediction.name == label:
        continue
    else:
        print(f"Id: {index}")
        print(f"Message: {message}")
        print(f"True Route: {label}, Predicted Route: {prediction.name}")
        print()

Id: 91
Message: Can you tell me about the gaming features of the Alienware laptop?
True Route: product_information, Predicted Route: None



In [ ]:
for (index, row), label in zip(X_test.iterrows(), y_test):
    message = row["Message"]
    prediction = oa_rl(message)

    if prediction.name == label:
        continue
    else:
        print(f"Id: {index}")
        print(f"Message: {message}")
        print(f"True Route: {label}, Predicted Route: {prediction.name}")
        print()

Id: 91
Message: Can you tell me about the gaming features of the Alienware laptop?
True Route: product_information, Predicted Route: None

Id: 78
Message: What is the storage capacity of the Samsung Galaxy Tab?
True Route: product_information, Predicted Route: None



## Save the Route Layer

To save our route layer we call the to_json method:

In [32]:
hf_rl.to_json("layer.json")

2024-11-19 11:19:45 INFO semantic_router.utils.logger Saving route config to layer.json


## Load the Route Layer

We can view the router file we just saved to see what information is stored.

In [ ]:
import json

with open("layer.json", "r") as f:
    layer_json = json.load(f)

print(layer_json)

{'encoder_type': 'openai', 'encoder_name': 'text-embedding-3-small', 'routes': [{'name': 'chitchat', 'utterances': ['How do you usually spend your evenings?', 'What’s your favorite way to stay active?', "What's the best concert you've ever been to?", 'Have you seen any good movies lately?', 'Have you ever tried meditation?', 'How do you feel about social media?', 'Do you have any fun plans for the weekend?', 'What’s a fun fact about yourself?', "What's your favorite type of music?", 'What’s your go-to comfort food?', "What's your favorite holiday?", "What's your favorite way to relax?", 'What’s something you’ve always wanted to learn?', 'Do you have any pets?', 'I love trying new recipes, do you cook?', "What's your favorite season of the year?", 'Do you like to play video games?', 'Do you prefer coffee or tea?', 'Have you traveled anywhere interesting recently?', "How's your day going?", 'What do you think about the weather today?', 'I enjoy hiking, do you like outdoor activities?', '

It tells us our encoder type, encoder name, and routes. This is everything we need to initialize a new router. To do so, we use the from_json method.

In [ ]:
rl = RouteLayer.from_json("layer.json")